In [24]:
import pandas as pd
import numpy as np
import os
os.listdir()
from sklearn.preprocessing import OneHotEncoder
import typing

In [25]:
df = pd.read_csv("medical_students_dataset.csv")
##https://www.kaggle.com/datasets/slmsshk/medical-students-dataset

In [26]:
df = df.dropna()
df.index = [i for i in range(len(df.index))]
df = df.iloc[:, 1:]

In [27]:
df ##  some preprocessing here!!!!  to fit the things into 

,Age,Gender,Height,Weight,Blood Type,BMI,Temperature,Heart Rate,Blood Pressure,Cholesterol,Diabetes,Smoking
0,32.0,Female,182.537664,55.741083,A,16.729017,98.260293,76.0,130.0,216.0,Yes,No
1,28.0,Male,186.489402,52.389752,AB,15.063921,98.227788,85.0,123.0,128.0,No,No
2,34.0,Female,182.416302,76.371050,AB,22.950992,98.118274,86.0,97.0,247.0,No,No
3,31.0,Male,166.489929,49.955569,B,18.022207,98.809750,82.0,96.0,223.0,No,No
4,29.0,Female,179.909041,90.679436,AB,28.015787,98.782269,81.0,108.0,227.0,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
50788,25.0,Female,156.297842,90.690186,B,37.123963,98.664591,70.0,132.0,164.0,Yes,No
50789,23.0,Female,162.884951,82.485778,B,31.089745,98.852347,65.0,94.0,188.0,No,Yes
50790,34.0,Male,165.651315,93.099756,A,33.928040,97.862209,62.0,100.0,205.0,No,No
50791,34.0,Male,161.590030,90.877589,B,34.803881,98.728836,70.0,96.0,208.0,No,No


In [28]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
non_numeric = ["Gender", "Blood Type", "Diabetes", "Smoking"]
numeric = {"Age":0, "Height":0, "Weight":0, "BMI":0, "Temperature":0, "Heart Rate":0, "Blood Pressure":0, "Cholesterol":0}
one_hotted = ["Blood Type"]

In [29]:
for feature in non_numeric:
    df[feature]= LabelEncoder().fit_transform(df[feature])

for feature in numeric.keys():
    mean = df[feature].mean()
    std = df[feature].std()
    numeric[feature] = mean, std
    df[feature] = (df[feature] - mean)/(std + 1e-5) 

In [30]:
df = df.drop("Blood Type", axis = 1)
#df_onehotted = OneHotEncoder(sparse_output=False).fit_transform(np.array(df["Blood Type"]).reshape(-1,1))

In [31]:
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch import nn as nn

In [32]:
def return_splitted(df:pd.DataFrame)->tuple[np.ndarray,]:
    df = df.sample(frac = 1)
    N = int(len(df)*0.8)
    train, test = map(np.array, [df.iloc[:N,:], df.iloc[N:, :]])
    return train, test
train, test = return_splitted(df)

In [33]:
import auto_encoder
from auto_encoder import auto_encoder_bb
from typing import Callable

In [34]:
df

,Age,Gender,Height,Weight,BMI,Temperature,Heart Rate,Blood Pressure,Cholesterol,Diabetes,Smoking
0,1.223955,0,0.530054,-0.823375,-0.940405,-0.677238,-0.304809,1.077073,0.841271,1,0
1,0.404662,1,0.803969,-1.016954,-1.176250,-0.742236,0.476180,0.590020,-1.493707,0,0
2,1.633602,0,0.521641,0.368253,-0.059123,-0.961220,0.562956,-1.219035,1.663820,0,0
3,1.019132,1,-0.582298,-1.157557,-0.757238,0.421460,0.215850,-1.288614,1.027007,0,0
4,0.609486,0,0.347850,1.194734,0.658255,0.366509,0.129074,-0.453666,1.133143,0,1
...,...,...,...,...,...,...,...,...,...,...,...
50788,-0.209807,0,-1.288765,1.195355,1.948339,0.131199,-0.825468,1.216231,-0.538489,1,0
50789,-0.619454,0,-0.832178,0.721452,1.093651,0.506638,-1.259350,-1.427772,0.098323,0,1
50790,1.633602,1,-0.640427,1.334536,1.495668,-1.473250,-1.519680,-1.010298,0.549398,0,0
50791,1.633602,1,-0.921936,1.206179,1.619723,0.259663,-0.825468,-1.288614,0.629000,0,0


In [35]:
#auto_encoder_bb([20, 50, 4])(torch.randn(10, 20), encoder_output = True)
F.one_hot(torch.tensor([0, 3, 1]), 5).shape

torch.Size([3, 5])

In [36]:
def loss(numerical_columns:list[int] = [], 
         binary_columns:list[int] = [],
         categorical_columns:list[tuple[float,float]] = [],
         )->Callable[[torch.Tensor, torch.Tensor],torch.Tensor]:
        #@torch.compile ## since we have a lot of for loops compiling may save some time!!!
        def temp_loss(X:torch.Tensor, y:torch.Tensor)->torch.Tensor:
            loss = nn.MSELoss()(X[:, numerical_columns], y[:, numerical_columns])
            for feature in categorical_columns:
                i, class_size = feature
                loss += F.cross_entropy(X[:, i:i+class_size], y[:, i:i+class_size])
            for i in binary_columns:
                loss += F.binary_cross_entropy_with_logits(X[:, i], y[:, i])               
            return loss
        return temp_loss


In [37]:
loss_fn = loss(numerical_columns = [0,2,3,4,5,6,7,8], binary_columns = [1,9,10])

In [38]:
torch_df = torch.tensor(df.to_numpy(), dtype = torch.float32)

In [50]:
model = auto_encoder_bb([11, 10, 9, 8])

In [51]:
from torch.optim import SGD, Adam

In [60]:
optimizer = Adam(model.parameters(), lr = 0.0001, weight_decay= 0.03)
for i in range(4000):
    optimizer.zero_grad()
    y_pred = model(torch_df)
    loss_ = loss_fn(y_pred, torch_df)
    loss_.backward()
    optimizer.step()
    if i % 100 == 0:
        print(f"The loss is now {loss_.item()}")    

The loss is now 0.12905265390872955
The loss is now 0.12891453504562378
The loss is now 0.12878529727458954
The loss is now 0.12863320112228394
The loss is now 0.12846817076206207
The loss is now 0.1282968372106552
The loss is now 0.12813040614128113
The loss is now 0.12796565890312195
The loss is now 0.12781086564064026
The loss is now 0.12766973674297333
The loss is now 0.12774401903152466
The loss is now 0.1274232119321823
The loss is now 0.12731020152568817
The loss is now 0.12770554423332214
The loss is now 0.1272200495004654
The loss is now 0.12701135873794556
The loss is now 0.12693004310131073
The loss is now 0.12686236202716827
The loss is now 0.12750761210918427
The loss is now 0.12672831118106842
The loss is now 0.12670493125915527
The loss is now 0.12676259875297546
The loss is now 0.12659768760204315
The loss is now 0.12680651247501373
The loss is now 0.126484677195549
The loss is now 0.12639930844306946
The loss is now 0.12640027701854706
The loss is now 0.127392038702964

In [70]:
model(torch_df[6, :], encoder_output = True)

tensor([-5.0901, 11.2220, -3.9230, -1.5144,  2.9147, -9.4183, -0.6910,  0.1435],
       grad_fn=<ViewBackward0>)

In [65]:
torch_df[6, :]

tensor([-0.4146,  1.0000,  0.8933, -1.7046, -1.6836,  0.5688, -1.6065, -1.3582,
         1.5312,  0.0000,  1.0000])